In [1]:
import pandas as pd

all_skills = 'skills/Skills.xlsx'
digital_skills = 'skills/Digital Skills.xlsx'
coursera_skills = 'coursera_skills/Coursera Skills To DB.xlsx'

In [2]:
def extract_skills(file_name):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    all_skills = []
    for key, df in dfs.items():
        all_skills.extend(df['Skill'].unique().tolist())
    df_skills = pd.DataFrame({'skill': all_skills})
    return df_skills.drop_duplicates()

def parent_check(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    for key, df in dfs.items():
        df_error = df.loc[~df['Parent'].isin(df_skills['skill'])].dropna()
        print(df_error['Parent'].unique().tolist())

def extract_relation(file_name, df_skills):
    xl_file = pd.ExcelFile(file_name)
    dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
    df_rel = pd.DataFrame(columns=['Skill', 'Parent'])
    skill_to_id_dict = df_skills.set_index('skill').to_dict()['skill_id']
    for key, df in dfs.items():
        df_rel = df_rel.append(df.replace(skill_to_id_dict))
    df_rel = df_rel.reset_index(drop=True)
    df_rel = df_rel.dropna()
    df_rel = df_rel.drop_duplicates()
    df_rel['Parent'] = df_rel['Parent'].astype(int)
    return df_rel

### Extract Skills

In [3]:
# All skills
df_s1 = extract_skills(all_skills)
df_s1.head()

,skill
0,Applied Science
1,Art
2,Business
3,Computer Science
4,Data Science


In [4]:
df_s1.describe(include='object')

,skill
count,2291
unique,2291
top,Numerical Integration
freq,1


In [5]:
# Digital skills
df_s2 = extract_skills(digital_skills)
df_s2.head()

,skill
0,Data Analysis
1,Machine Learning
2,Algorithm
3,Artificial Intelligence
4,Big Data


In [6]:
df_s2.describe(include='object')

,skill
count,984
unique,984
top,Social Engineering
freq,1


In [7]:
# Combine
df_skills = df_s1.append(df_s2)
df_skills = df_skills.drop_duplicates()
df_skills = df_skills.dropna()
df_skills.describe()

,skill
count,3095
unique,3095
top,Web Framework
freq,1


In [8]:
df_skills['skill_id'] = df_skills.groupby('skill').grouper.group_info[0] + 1
df_skills = df_skills.reset_index(drop=True)
df_skills = df_skills[['skill_id', 'skill']]
df_skills.head()

,skill_id,skill
0,150,Applied Science
1,163,Art
2,322,Business
3,548,Computer Science
4,714,Data Science


In [9]:
df_skills.describe(include='all')

,skill_id,skill
count,3095.000000,3095
unique,NaN,3095
top,NaN,Web Framework
freq,NaN,1
mean,1548.000000,NaN
std,893.593867,NaN
min,1.000000,NaN
25%,774.500000,NaN
50%,1548.000000,NaN
75%,2321.500000,NaN


### Coursera Skills

In [10]:
df_c = pd.read_excel(coursera_skills)
df_c.head()

,skill_id,c_skill,Skill,Parent
0,457,Low Poly,Low Poly,3D Computer Graphics
1,462,Polygon Mesh,Polygon Mesh,3D Computer Graphics
2,466,Uv Mapping,Uv Mapping,3D Computer Graphics
3,467,Framing,Framing,3D Computer Graphics
4,469,3D Graphics Software,3D Graphics Software,3D Computer Graphics


In [11]:
df_coursera = df_c[['skill_id', 'c_skill']]
df_coursera = df_coursera.rename(columns={'c_skill': 'skill'})
df_coursera.head()

,skill_id,skill
0,457,Low Poly
1,462,Polygon Mesh
2,466,Uv Mapping
3,467,Framing
4,469,3D Graphics Software


In [12]:
df_others = df_c[['Skill']].rename(columns={'Skill': 'skill'})
df_others = df_others.drop_duplicates()
df_others = df_others.dropna()
df_others['skill_id'] = df_others.index + df_skills['skill_id'].max()
df_others.head()

,skill,skill_id
0,Low Poly,3095
1,Polygon Mesh,3096
2,Uv Mapping,3097
3,Framing,3098
4,3D Graphics Software,3099


In [25]:
# Combine
df_all = df_skills.append(df_others)
df_all = df_all.drop_duplicates(subset=['skill'])
df_all = df_all.reset_index(drop=True)
df_all = df_all[['skill_id', 'skill']]
df_all.head()

,skill_id,skill
0,150,Applied Science
1,163,Art
2,322,Business
3,548,Computer Science
4,714,Data Science


In [26]:
df_all.describe(include='all')

,skill_id,skill
count,9487.000000,9487
unique,NaN,9487
top,NaN,Analytics
freq,NaN,1
mean,5167.276589,NaN
std,3085.318469,NaN
min,1.000000,NaN
25%,2372.500000,NaN
50%,5267.000000,NaN
75%,7901.500000,NaN


### Data Checking/Cleaning

In [27]:
parent_check(all_skills, df_skills)

[]
[]
[]
[]
[]
[]
[]
[]
[]


In [28]:
parent_check(digital_skills, df_skills)

[]
[]
[]
[]
[]
[]


In [29]:
parent_check(coursera_skills, df_all)

[]


### Extract Skill Relation

In [30]:
# Skills
df_r1 = extract_relation(all_skills, df_skills)
df_r2 = extract_relation(digital_skills, df_skills)
df_rel = df_r1.append(df_r2)
df_rel = df_rel.drop_duplicates()
df_rel = df_rel.reset_index(drop=True)
df_rel = df_rel.rename(columns={'Skill': 'skill_1_id', 'Parent': 'skill_2_id'})
df_rel.head()

,skill_1_id,skill_2_id
0,1571,1880
1,2148,1880
2,2831,548
3,2831,1700
4,69,150


In [31]:
df_rel.describe(include='all')

,skill_1_id,skill_2_id
count,3750.0,3750.000000
unique,3087.0,NaN
top,1915.0,NaN
freq,7.0,NaN
mean,NaN,1440.435733
std,NaN,872.062332
min,NaN,1.000000
25%,NaN,699.750000
50%,NaN,1550.000000
75%,NaN,2186.500000


In [32]:
# Include coursera skills
df_r3 = extract_relation(coursera_skills, df_all)
# Combine
df_rel_all = df_rel.append(df_r3[['Skill', 'Parent']])
df_rel_all = df_rel_all.drop_duplicates()
df_rel_all = df_rel_all.reset_index(drop=True)
df_rel_all = df_rel_all.rename(columns={'Skill': 'skill_1_id', 'Parent': 'skill_2_id'})
df_rel_all.head()

,skill_1_id,skill_2_id,skill_1_id,skill_2_id
0,1571,1880.0,NaN,NaN
1,2148,1880.0,NaN,NaN
2,2831,548.0,NaN,NaN
3,2831,1700.0,NaN,NaN
4,69,150.0,NaN,NaN


In [33]:
df_rel_all.describe(include='all')

,skill_1_id,skill_2_id,skill_1_id,skill_2_id
count,3750.0,3750.000000,2349.0,2349.000000
unique,3087.0,NaN,2343.0,NaN
top,1915.0,NaN,4880.0,NaN
freq,7.0,NaN,2.0,NaN
mean,NaN,1440.435733,NaN,1689.597275
std,NaN,872.062332,NaN,1331.621092
min,NaN,1.000000,NaN,4.000000
25%,NaN,699.750000,NaN,775.000000
50%,NaN,1550.000000,NaN,1581.000000
75%,NaN,2186.500000,NaN,2285.000000


### Extract Coursera Relation

In [34]:
df_crel = df_c[['skill_id', 'Skill']].rename(columns={'skill_id': 'coursera_skill_id', 'Skill': 'skill'})\
    .merge(df_all, how='left').drop(columns=['skill'])
df_crel.head()

,coursera_skill_id,skill_id
0,457,3095
1,462,3096
2,466,3097
3,467,3098
4,469,3099


In [35]:
df_crel.describe(include='all')

,coursera_skill_id,skill_id
count,7301.000000,7301.000000
mean,3650.941652,6249.237776
std,2107.856673,2657.828696
min,0.000000,2.000000
25%,1826.000000,4283.000000
50%,3651.000000,6494.000000
75%,5476.000000,8491.000000
max,7301.000000,10395.000000


### Export

In [36]:
df_skills.to_csv('skills_db/skill.csv', index=False)
df_rel.to_csv('skills_db/skill_tree.csv', index=False)
df_all.to_csv('skills_db/all_skill.csv', index=False)
df_rel_all.to_csv('skills_db/all_skill_tree.csv', index=False)
df_coursera.to_csv('skills_db/coursera_skill.csv', index=False)
df_crel.to_csv('skills_db/coursera_map.csv', index=False)